# Bibliotek

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skyfield.api import EarthSatellite, load
from datetime import datetime, timedelta
from PIL import Image
import plotly.graph_objects as go

# Funksjoner og Konstanter

In [ ]:
earth_radius = 6371 #km

def sphere(size, texture): 
    N_lat = int(texture.shape[0])
    N_lon = int(texture.shape[1])
    theta = np.linspace(0,2*np.pi,N_lat)
    phi = np.linspace(0,np.pi,N_lon)
    
    # Set up coordinates for points on the sphere
    x0 = size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(N_lat),np.cos(phi))
    
    return x0, y0, z0

def create_satellite(tle_line1, tle_line2):
    return EarthSatellite(tle_line1, tle_line2)

def propagate_orbit(satellite, start_time, end_time, time_step):
    ts = load.timescale()
    times = ts.utc(start_time.year, start_time.month, start_time.day, 
                   start_time.hour, start_time.minute, start_time.second + np.arange(0, int((end_time - start_time).total_seconds()), time_step))
    geocentric = satellite.at(times)
    positions = geocentric.position.km
    velocities = geocentric.velocity.km_per_s
    return positions, velocities

def plot_orbits(positions_list, velocities_list, labels):
    fig = go.Figure()

    # Plot the Earth
    texture = np.asarray(Image.open('temp.jpg')).T
    colorscale =[[0.0, 'rgb(30, 59, 117)'],
                 [0.1, 'rgb(46, 68, 21)'],
                 [0.2, 'rgb(74, 96, 28)'],
                 [0.3, 'rgb(115,141,90)'],
                 [0.4, 'rgb(122, 126, 75)'],
                 [0.6, 'rgb(122, 126, 75)'],
                 [0.7, 'rgb(141,115,96)'],
                 [0.8, 'rgb(223, 197, 170)'],
                 [0.9, 'rgb(237,214,183)'],
                 [1.0, 'rgb(255, 255, 255)']]              
    xs, ys, zs = sphere(earth_radius, texture)
    fig.add_trace(go.Surface(x=xs, y=ys, z=zs,
                      surfacecolor=texture,
                      colorscale=colorscale,
                      showscale=False,
                      hoverinfo='none',
                      name='Earth',
                      showlegend=True)
    )
    
    # Plot each satellite's orbit with different colors from the colormap
    cmap = plt.get_cmap('cool')  # Use the 'cool' colormap
    for i, (positions, velocities, label) in enumerate(zip(positions_list, velocities_list, labels)):
        color = cmap(i / len(positions_list))  # Get a color from the colormap
        hover_text = [f"Velocity: {np.linalg.norm(vel):.2f} km/s" for vel in velocities.T]
        fig.add_trace(go.Scatter3d(
            x=positions[0], y=positions[1], z=positions[2],
            mode='lines',
            name=label,
            line=dict(color='rgb({}, {}, {})'.format(*[int(c * 255) for c in color[:3]])),
            text=hover_text,
            hoverinfo='text'
        ))

    # Set the layout for the plot
    fig.update_layout(
        scene=dict(
            xaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='x (km)'),
            yaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='y (km)'),
            zaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='z (km)'),
            bgcolor='black'
        ),
        legend=dict(
            x=0, y=1,  # Position the legend at the bottom right
            xanchor='left', yanchor='top'
        ),
        margin=dict(r=0, l=0, b=0, t=0)
    )

    # Show the plot
    fig.show()

# Analyse

In [ ]:
tle_lines = [
    # ISS (LEO)
    (('1 25544U 98067A   20334.54791667  .00016717  00000-0  10270-3 0  9000', 
      '2 25544  51.6442  21.4614 0001448  45.3583  314.7000 15.49112345  9000'), 'ISS (LEO)'),
    # GPS BIIR-2 (MEO)
    (('1 24876U 97035A   20334.54791667  .00000023  00000-0  00000-0 0  9993',
      '2 24876  54.8990  56.0000 0000001  56.0000  56.0000  2.00562500  9993'), 'GPS BIIR-2 (MEO)'),
    # Molniya 1-93 (HEO)
    (('1 24779U 97017A   20334.54791667  .00000023  00000-0  00000-0 0  9993',
      '2 24779  62.8000  56.0000 0000001  56.0000  56.0000  2.00562500  9993'), 'Molniya 1-93 (HEO)'),
    # SES-1 (Geostationary)
    (('1 36516U 10018A   20334.54791667  .00000023  00000-0  00000-0 0  9993',
      '2 36516   0.0180  92.0000 0000001  92.0000 268.0000  1.00270000  9993'), 'SES-1 (GEO)')
]

start_time = datetime.now()
end_time = start_time + timedelta(hours=48)
time_step = 60  # seconds

positions_list = []
velocities_list = []
labels = []
for (tle_line1, tle_line2), label in tle_lines:
    satellite = create_satellite(tle_line1, tle_line2)
    positions, velocities = propagate_orbit(satellite, start_time, end_time, time_step)
    positions_list.append(positions)
    velocities_list.append(velocities)
    labels.append(label)

plot_orbits(positions_list, velocities_list, labels)